In [ ]:
import numpy as np
import pandas as pd 
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Conv2D,MaxPool2D,Dense,ZeroPadding2D
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import random
from PIL import Image 

**EDA**

In [ ]:
path = "../input/100-bird-species/train"
category = []
count = []
img_path = []
class_ = []
for folder in os.listdir(path):
    i = 0
    category.append(folder)
    for subfolder in os.listdir(os.path.join(path,folder)):
        i += 1
        img_path.append(os.path.join(path,folder,subfolder))
        class_.append(folder)
    count.append(i)
df = pd.DataFrame()
df["specie_category"] = category
df["count"] = count


In [ ]:
sns.countplot(x = df["specie_category"])
plt.plot()

**Though the name of species is not clearly visible(there are 400 of them), however the train dataset seems to be balanced**

In [ ]:
plt.figure(1, figsize=(25, 25))

n = 0
for i in range(16):
    j = random.randint(0,2358)
    n += 1
    plt.subplot(4, 4, n)
    random_img = img_path[j]
    imgs = cv2.imread(random_img)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(imgs, cmap=plt.cm.binary)
    plt.xlabel(class_[j])
plt.show()

**Using a basic Convolution Neural Network**

In [ ]:
train_dir = '../input/100-bird-species/train'
#Object of ImageDataGenrator, can specify data augmentation steps here
train_datagen = ImageDataGenerator(rescale = 1.0/255,
                                          width_shift_range=0.2,
                                          height_shift_range=0.2,
                                          zoom_range=0.2,
                                          horizontal_flip=True,
                                          fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(train_dir,batch_size=32,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))
test_dir = '../input/100-bird-species/test'
#Object of ImageDataGenrator, can specify data augmentation steps here
test_datagen = ImageDataGenerator(rescale = 1.0/255,
                                          width_shift_range=0.2,
                                          height_shift_range=0.2,
                                          zoom_range=0.2,
                                          horizontal_flip=True,
                                          fill_mode='nearest')
test_generator = test_datagen.flow_from_directory(test_dir,batch_size=32,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))

In [ ]:
model = Sequential([
    Conv2D(32,3,activation='relu',input_shape=(150, 150,3)),
    MaxPool2D(pool_size=2,strides=2),
    ZeroPadding2D(padding=(2,2)),
    Conv2D(64,3,activation='relu'),
    MaxPool2D(pool_size=2,strides=2),
    Flatten(),
    Dense(128,activation="relu"),
    Dense(400,activation="softmax")
])

In [ ]:
model.compile("adam",loss="categorical_crossentropy",metrics=["accuracy"])


In [ ]:
history = model.fit(train_generator,validation_data = test_generator,epochs=10,batch_size=256)